## Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### Graphic libraries
import matplotlib.pyplot as plt
import seaborn as sns 
from fastai.tabular.all import *
import numpy as np
import pandas as pd
from pathlib import Path

path = Path('../input/tabular-playground-series-jan-2021')

## Reading data

In [ ]:
train_df = pd.read_csv(path/'train.csv')
train_df.pop('id')
train_df_len = len(train_df)

test_df = pd.read_csv(path/'test.csv')
test_ID_codes_df = test_df.pop('id')

In [ ]:
# valid_idx = range(round(0.99*train_df_len), train_df_len)
# data: DataBunch = TabularDataBunch.from_df(path, train_df, dep_var='target', valid_idx=valid_idx, test_df=test_df)
# learn = tabular_learner(data, layers=[200,100], metrics=accuracy, path='.')


## Import TabNet

In [ ]:
!pip install pytorch-tabnet
import pandas as pd
import numpy  as np
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold

In [ ]:
# model = TabNetRegressor(n_d=16,
#                        n_a=16,
#                        n_steps=4,
#                        gamma=1.9,
#                        n_independent=4,
#                        n_shared=5,
#                        seed=seed,
#                        optimizer_fn = torch.optim.Adam,
#                        scheduler_params = {"milestones": [150,250,300,350,400,450],'gamma':0.2},
#                        scheduler_fn=torch.optim.lr_scheduler.MultiStepLR)

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test_data  = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
sample     = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')
# solution   = pd.read_csv('../input/house-prices-advanced-regression-solution-file/solution.csv')

In [ ]:
features = train_data.columns[1:-1]

In [ ]:
X      = train_data[features]
y      = np.log1p(train_data["target"])
X_test = test_data[features]
# y_true = solution["SalePrice"]

In [ ]:
# X      =      X.apply(lambda x: x.fillna(x.mean()),axis=0)
# X_test = X_test.apply(lambda x: x.fillna(x.mean()),axis=0)

In [ ]:
X      = X.to_numpy()
y      = y.to_numpy().reshape(-1, 1)
X_test = X_test.to_numpy()

## Train

In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in kf.split(X):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    regressor = TabNetRegressor(verbose=1,seed=42)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=1, max_epochs=2,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(np.expm1(regressor.predict(X_test)))

predictions = np.mean(predictions_array,axis=0)

## CV score

In [ ]:
print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )